### Music Recommendation System

To make it works, firstly you need to have Model.h5 in MODEL_DIR_NAME or run train.py!

Remember to put test mp3 files in TEST_DATASET_DIR_NAME

In [1]:
from utils.spectogram import create_test_spectograms, split_test_spectogram


create_test_spectograms()
split_test_spectogram()

NameError: name 'create_spectograms' is not defined

In [3]:
from utils.constants import MODEL_DIR_NAME
from keras.models import Model, load_model
from utils.spectogram import load_splited_images
import numpy as np

loaded_model = load_model(f'{MODEL_DIR_NAME}/Model.h5')
loaded_model.set_weights(loaded_model.get_weights())
matrix_size = loaded_model.layers[-2].output.shape[1]
new_model = Model(loaded_model.inputs, loaded_model.layers[-2].output)
print(new_model.summary())

images, labels = load_splited_images()
images = np.expand_dims(images, axis=3)
images = images / 255.
print(np.unique(labels))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 122, 122, 64)      3200      
                                                                 
 batch_normalization (BatchN  (None, 122, 122, 64)     256       
 ormalization)                                                   
                                                                 
 average_pooling2d (AverageP  (None, 61, 61, 64)       0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 128)       401536    
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 128)      512   

In [4]:

recommend_source = input("Enter name of song to which you will be looking for similar ones: ")
prediction_anchor = np.zeros((1, matrix_size))
count = 0
predictions_song = []
predictions_label = []
counts = []
distance_array = []

for i in range(0, len(labels)):
    if(labels[i] == recommend_source):
        test_image = images[i]
        test_image = np.expand_dims(test_image, axis=0)
        prediction = new_model.predict(test_image)
        prediction_anchor = prediction_anchor + prediction
        count = count + 1
    elif(labels[i] not in predictions_label):
        predictions_label.append(labels[i])
        test_image = images[i]
        test_image = np.expand_dims(test_image, axis=0)
        prediction = new_model.predict(test_image)
        predictions_song.append(prediction)
        counts.append(1)
    elif(labels[i] in predictions_label):
        index = predictions_label.index(labels[i])
        test_image = images[i]
        test_image = np.expand_dims(test_image, axis=0)
        prediction = new_model.predict(test_image)
        predictions_song[index] = predictions_song[index] + prediction
        counts[index] = counts[index] + 1
prediction_anchor = prediction_anchor / count
for i in range(len(predictions_song)):
    predictions_song[i] = predictions_song[i] / counts[i]
    distance_array.append(np.sum(prediction_anchor * predictions_song[i]) / (np.sqrt(np.sum(prediction_anchor**2)) * np.sqrt(np.sum(predictions_song[i]**2))))

distance_array = np.array(distance_array)
recommendations = 0

print("Recommendation is:")


# Number of Recommendations is set to 2.
i = 0
recommended = []

while(i < 3):
    index = np.argmax(distance_array) # Index piosenki o najwyższym podobieństwie
    value = distance_array[index] # Wartośc rekomendacji
    input_name = recommend_source[recommend_source.find('_') + 1:] # nazwa piosenki od usera
    
    recommendation = predictions_label[index]
    song_name = recommendation[recommendation.find('_') + 1:] # Nazwa piosenki zarekomendowanej

    distance_array[index] = -np.inf # Ustawienie tej wartości na -inf

    if song_name in recommended:
        continue
    
    recommended.append(song_name)

    if song_name == input_name:
        continue

    print(f'Song Name: {song_name} with value = {value}')
    i += 1



1/1 [==============================] - 0s 24ms/step
Recommendation is:
Song Name: 30s_06 Moves with value = 0.9990768243543222
Song Name: 30s_04 No Favors (feat. Eminem) with value = 0.9984790286062282
Song Name: 30s_05 Jump Out the Window with value = 0.9981257089400036
